In [1]:
import numpy as np
import scipy.linalg as sln
import matplotlib.pyplot as plt
import warnings
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

Points
<br># -> Doc(Main)
<br>## -> My Notes
<br>### -> Missing, Corrections to do

In [2]:
def mv(A, v, transp_flag):
    if transp_flag == 0:
        return A @ v
    else:
        return A.T @ v

In [3]:
def unv(j, n):
    e = np.zeros(n)
    e[j] = 1
    return e

In [4]:
def element(A, i=None, j=None):
    # Handling default arguments
    if i is None:
        i = slice(None)  # Selecting all rows
    if j is None:
        j = slice(None)  # Selecting all columns

    if isinstance(A, np.ndarray):  # Matrix
        if min(A.shape) > 1:  # Check if A is not a vector
            if isinstance(i, list) and isinstance(j, list):
                e = A[np.ix_(i, j)]
            else:
                e = A[i, j]
        else:
            e = A[i] if isinstance(i, list) else A[i]
    
    return e

In [5]:
def krylov_ata(A, v1=None, k=10, full=1, reortho=2):
    if v1 is None:
        v1 = np.random.randn(A.shape[1])

    k = min(k, min(A.shape))

    alpha = np.zeros(k)
    beta = np.zeros(k if full else k-1)

    len_v1 = len(v1)
    if reortho:
        V = np.zeros((len_v1, k + 1))
        V[:, 0] = v1 / np.linalg.norm(v1)
        #U = np.zeros((A.shape[0], k))
    else:
        v = v1 / np.linalg.norm(v1)

    for j in range(k):
        if reortho:
            r = A @ V[:, j]
            if j == 0 and reortho == 2:
                U = np.zeros((len(r), k))
        else:
            r = A @ v

        if j > 0:
            if reortho == 2:
                r -= beta[j-1] * U[:, j-1]
                r -= U[:, :j] @ (U[:, :j].T @ r)
            else:
                r -= beta[j-1] * u
        alpha[j] = np.linalg.norm(r)
        if alpha[j] == 0:
            break

        if reortho == 2:
            U[:, j] = r / alpha[j]
            r = A.T @ U[:, j]
        else:
            u = r / alpha[j]
            r = A.T @ u

        if reortho:
            r -= alpha[j] * V[:, j]
            r -= V[:, :j+1] @ (V[:, :j+1].T @ r)
        else:
            r -= alpha[j] * v

        if j < k - 1 or full:
            beta[j] = np.linalg.norm(r)
            if beta[j] == 0:
                break

            if reortho:
                V[:, j+1] = r / beta[j]
            else:
                v = r / beta[j]

    if not reortho:
        V = v
    if reortho < 2:
        U = u
    
    return V, U, alpha, beta

In [35]:
def krylov_ata_expand(A, V, U, c, k=10):
    m = V.shape[1]
    V = np.concatenate((V, np.zeros((V.shape[0], k))), axis=1)
    U = np.concatenate((U, np.zeros((U.shape[0], k))), axis=1)
    alpha = np.zeros(k)
    beta = np.zeros(k)
    

    for j in range(m, k + m):
        if j == m:
            r = A @ V[:, j - 1] - (U[:, :j - 1] @ c.T)
        else:
            r = A @ V[:, j - 1] - beta[j - m - 1] * U[:, j - 2]

        r -= - U[:, :j - 1] @ (U[:, :j - 1].T @ r)
        alpha[j - m] = np.linalg.norm(r)
        if alpha[j - m] == 0:
            break
        U[:, j - 1] = r / alpha[j - m]
        r = A.T @ U[:, j - 1] - alpha[j - m] * V[:, j - 1]
        r -= V[:, :j] @ (V[:, :j].T @ r)
        beta[j - m] = np.linalg.norm(r)
        if beta[j - m] == 0:
            break
        V[:, j] = r / beta[j - m]

    return V, U, alpha, beta

In [36]:
def krylov_schur_svd(A, v1 = None, nr = 1, tol = 1e-6, absrel = 'rel', mindim = 10, maxdim = 20, maxit = 1000, target = np.inf, info = 1):
    if v1 is None:
        v1 = np.random.rand(A.shape[1])
    if mindim < nr:
        mindim = nr
    if maxdim < 2 * mindim:
        maxdim = 2 * mindim
    if absrel == 'rel':
        tol *= np.linalg.norm(A, 1)

    B = np.zeros((maxdim, maxdim + 1))
    print(1)
    # Slow Here
    V, U, alpha, beta = krylov_ata(A, v1, mindim)
    print(2)
    # Bidiagonal Form for the first mindim rows and cols
    B[:mindim + 1, :mindim + 1] = np.diag(np.append(alpha, [0])) + np.diag(beta, 1)
    hist = np.zeros(maxit, dtype=np.float64)    
    np.set_printoptions(precision=12) 
    # Modified MATLAB code ordering
    print(3)
    # Slow Here
    v, u, a, b = krylov_ata_expand(A, V, U, B[:mindim, mindim], maxdim - mindim)
    print(4)
    for k in range(maxit):
        V, U, alpha, beta = v.copy(), u.copy(), a.copy(), b.copy()
        print(5)
        B[mindim: maxdim, mindim: maxdim] = np.diag(alpha) + np.diag(beta[:maxdim - mindim - 1], 1)        
        B[maxdim - 1, maxdim] = beta[maxdim - mindim - 1]
        X, sigma, Y = np.linalg.svd(B[:maxdim, :maxdim])
        # Restart of Lanczos algorithm
        V = np.concatenate((element(V[:, :maxdim] @ Y, list(range(V.shape[0])), list(range(mindim))), V[:, maxdim:maxdim + 1]), axis=1)
        U = element(U[:, :maxdim] @ X, list(range(U.shape[0])), list(range(mindim)))    
        c = B[:, maxdim]
        e = (c @ X)[:mindim]
        B[:mindim, :mindim + 1] = np.concatenate((np.diag(sigma[:mindim]), e.reshape(-1, 1)), axis=1)
        err = np.linalg.norm(e[:nr])
        hist[k] = err
        
        if info:
            print(str(k) + ": " + str(hist[k]))
        if err < tol:
            sigma = sigma[:nr]
            V = V[:, :nr]
            U = U[:, :nr]
            mvs = np.arange(1, k + 1) * (maxdim - mindim) + mindim
            print(f"Found after {k + 1} iteration(s) with residual = {err}")
            return sigma, V, U, hist[:k+1], mvs
    
    mvs = 2 * (np.arange(1, k + 1) * (maxdim - mindim) + mindim)
    if info:
        print(f"Quit after max {k + 1} iterations with residual = {err}")
    sigma = sigma[:mindim]
    V = V[:, :mindim]
    return sigma, V, U, hist, mvs

Tests

In [37]:
A = np.random.rand(200,100)
A

array([[0.969729606461, 0.099939488277, 0.475138725312, ...,
        0.089164066974, 0.910050030327, 0.183075573985],
       [0.336677168703, 0.156113881088, 0.243100479574, ...,
        0.79709447475 , 0.603196941122, 0.451571149457],
       [0.553933640368, 0.472197650532, 0.670329690889, ...,
        0.900826068948, 0.470876634193, 0.901086688383],
       ...,
       [0.660223023543, 0.705177430802, 0.51441085113 , ...,
        0.708047296392, 0.568332350147, 0.626892671076],
       [0.724034421146, 0.112353874989, 0.619100114312, ...,
        0.972962570302, 0.9444677461  , 0.625275023164],
       [0.966934969849, 0.351894516185, 0.612715041593, ...,
        0.485370222748, 0.412303126184, 0.387705513893]])

In [38]:
sigma, V, U, hist, mvs = krylov_schur_svd(A, info=2,tol=1e-200,maxit= 7,nr= 3)

1
2
3
4
5
0: 0.017681027645471564
5
1: 0.0010374076813990248
5
2: 3.8828556628942295e-05
5
3: 1.2793498951597756e-06
5
4: 4.107483488449618e-08
5
5: 1.282132414399532e-09
5
6: 3.980081140864027e-11
Quit after max 7 iterations with residual = 3.980081140864027e-11


In [40]:
hist

array([1.768102764547e-02, 1.037407681399e-03, 3.882855662894e-05,
       1.279349895160e-06, 4.107483488450e-08, 1.282132414400e-09,
       3.980081140864e-11])

In [41]:
mvs

array([ 40,  60,  80, 100, 120, 140])